# Parallel view performance test 

## Setup

In [54]:
import uuid
import random
import requests
import json

from cloudant import cloudant, Cloudant
from getpass import getpass

In [35]:
cloudant_username = raw_input('Username: ')

Username: physion-testy016


In [36]:
cloudant_password = getpass('Password: ')

Password: ········


## Fixture

In [19]:
n_users = int(input('Users: '))

Users: 1000


In [26]:
n_docs = n_users * 1000

In [27]:
teams = [str(uuid.uuid4()) for x in xrange(n_users * 10)]

In [28]:
owners = [str(uuid.uuid4()) for x in xrange(n_users)]

In [29]:
ids = [str(uuid.uuid4()) for x in xrange(n_docs)]

In [55]:
with open('fixture.json', 'w') as outfile:
    json.dump({'ids': ids, 'teams': teams, 'users': owners}, outfile)

In [51]:
def random_subset(col, num=1):
    indices = random.sample(range(len(col)), num)
    return [col[i] for i in sorted(indices)]

In [58]:
test_db = 'test'

with cloudant(cloudant_username, cloudant_password, account=cloudant_username) as client:
    try:
        db = client[test_db]
    except KeyError:
        db = client.create_database(test_db)
    
    for id in ids:
        doc = {
            '_id': id,
            'owner': random_subset(owners, 1)[0],
            'attributes': {
                'foo': 'bar',
                'baz': 10
            },
            'links': {
                '_collaboration_roots': random_subset(teams, 3)
            }
        }
        
        db.create_document(doc)

HTTPError: 409 Client Error: Conflict for url: https://physion-testy016.cloudant.com/test